In [1]:
print('True')

True


In [2]:
# importing os and load_dotenv
import os
from dotenv import load_dotenv

In [3]:
# loading dotenv and getting the openai api key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# getting the openai key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
# importing PyPDFLoader from langchain.document_loaders
from langchain.document_loaders import PyPDFLoader

In [6]:
# showing the present working directory
%pwd

'd:\\genai_project\\Interview-Questions-Generator\\research'

In [7]:
%cd ..

d:\genai_project\Interview-Questions-Generator


d:\genai_project\Interview-Questions-Generator\inter\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
%pwd

'd:\\genai_project\\Interview-Questions-Generator'

# Step 1: Ingesting Data

In [9]:
# path of the file daved into a variable
# calling the PyPDFLoader() class and creating an object
# and through the object call a function call functioned as load()
file_path = 'data/stats.pdf'
loader = PyPDFLoader(file_path)
data = loader.load()

In [10]:
# showing the whole dataset
data
# showing the data through indexing
data[0]
# length of the dataset
len(data)
# printint the dataset through length wise
for i in data:
    print(i.page_content)

Importance
and
the
use
of
correlation
in
Statistics
Introduction
Correlation
is
a
statistical
measure
that
expresses
the
extent
to
which
two
variables
are
linearly
related.
It
is
a
common
tool
for
describing
simple
relationships
without
making
a
statement
about
cause
and
effect.
Correlation
coefficients
range
from
-1
to
1,
with
a
value
of
0
indicating
no
linear
relationship
between
the
two
variables,
a
value
of
1
indicating
a
perfect
positive
linear
relationship,
and
a
value
of
-1
indicating
a
perfect
negative
linear
relationship.
Correlation
is
important
in
statistics
because
it
can
be
used
to
1.
Identify
relationships
between
variables:
Correlation
can
be
used
to
identify
whether
there
is
a
relationship
between
two
variables,
and
if
so,
whether
the
relationship
is
positive
or
negative.
This
information
can
be
useful
for
understanding
the
relationships
between
different
factors
in
a
complex
system.
2.
Make
predictions:
If
there
is
a
strong
correlation
between
two
variables,
it
is
poss

In [11]:
# saving the all page content of the whole dataset into a string varible
question_gen = ''

for page in data:
    question_gen += page.page_content

# Step-2 : Chunking 
#### means partioning text into chunking giving chunk_size and chunk_overlap

In [12]:
# importing TokenTextSplitter from langchain.text_splitter. we need it for
# splitting the whole documents because every model has specific token limitations
# or we can say chunk_size
from langchain.text_splitter import TokenTextSplitter

In [13]:
# now we will call the TokenTextSplitter() class and making an object of it
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 1000,
    chunk_overlap = 200
)

In [14]:
# calling the function split_text() through the object we just created
# and passing the question_gen where all data are saved into single string varible.
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)
# now our whole dataset saved into a list inside the string type data

In [15]:
# importing Document from langchain.docstore.document
from langchain.docstore.document import Document

In [16]:
# now we need to convert the data into documets type
# and this is the procedure to convert the data into a list
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]


In [17]:
# creating an obejct of TokenTextSplitter() class
# why we are doing the same thing because if we check the previous splitter,
# the length of the list is 1
# and we use 10000 for chunk size.
# now we will take smaller chunks size and splitting the text
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 100,
    chunk_overlap = 50
)

In [18]:
# calling split_documents() function through the object splitter_ans_gen
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [19]:
# getting first index of the documents through the page_content attribute
document_answer_gen[0].page_content

'Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom'

In [20]:
# length of the document
len(document_answer_gen)

30

In [21]:
# importing ChatOpenAI from from langchain.chat_models
from langchain.chat_models import ChatOpenAI

In [22]:
# remove warnings while running the cell
import warnings
warnings.filterwarnings('ignore')

In [23]:
# creating an object of ChatOpenAI() class
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

In [24]:
# prompt template designing
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [25]:
# importing PromptTemplate from langchain.prompts
from langchain.prompts import PromptTemplate

In [26]:
# calling the function PromptTemplate() and passing few parameters 
# and saved into a varible called as PROMPT_QUESTIONS
PROMPT_QUESTIONS = PromptTemplate(
    template=prompt_template, 
    input_variables=['text']
    )

In [27]:
# this prompt template is designed for another concept of refinig or making meaningful answers
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [28]:
# calling the function PromptTemplate() and passing few parameters 
# and saved into a varible called as REFINE_PROMPT_QUESTIONS
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [29]:
# importing load_summarize_chain from langchain.chains.summarize
# here load_summarize_chain would be either class or function
# and langchain.chains.summarize is coming from the path langchain/chains/summarize
from langchain.chains.summarize import load_summarize_chain

In [30]:
# calling load_summarize_chain() function and passing few parameters
ques_gen_chain = load_summarize_chain(
    llm = llm_ques_gen_pipeline, 
    chain_type = "refine", 
    verbose = True, 
    question_prompt=PROMPT_QUESTIONS, 
    refine_prompt=REFINE_PROMPT_QUESTIONS
    )

In [31]:
# these cell will not work because of openai 429 quota issues.
# if you want to access this cell, u need to have paid openai account.
ques = ques_gen_chain.run(document_ques_gen)
print(ques)

## Step-3 : Embeddings
#### we used here OpenAIEmbeddings

In [32]:
# importing OpenAIEmbeddings from langchain.embeddings.openai
# the path is langchain/embeddings/openai and 
# OpenAIEmbeddings is either class or function
from langchain.embeddings.openai import OpenAIEmbeddings

In [33]:
# creating an object of OpenAIEmbeddings() class
embeddings=OpenAIEmbeddings()

## Step-4 : Data stored into Vector Daabase
#### texts are converted into numbers and these numbers are called vectors


In [34]:
# for storing the vectors, we need to have a vector database and here we
# used FAISS vector database system and this is offline database. 
# it used for efficient similarity search and clustering of large datasets of vectors.
# importing FAISS and this is a class
from langchain.vectorstores import FAISS

In [ ]:
# creating object as vector_store of  FAISS class 
# and calling a function through of it.
# passing params as document_answer_gen and embeddings model
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

In [36]:
# since we need to pass the answers to the ChatOpenAI module.
# so we call the ChatOpenAI() module
llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [ ]:
# this is my question
ques

# we need to see the question as list
# we split the question through split() function and we splitted by \n
ques_list = ques.split("\n")

# here are the questions and sshowing inside a list
ques_list

In [38]:
# now we need answers for those question that just created. For this
# reason, we need ti import RetrievalQA module from the path langchain.chains
from langchain.chains import RetrievalQA

In [ ]:
# creating object of RetrievalQA() class and 
# calling function from_chain_type() through the object and 
# passing few parameters.
answer_generation_chain = RetrievalQA.from_chain_type(
    llm=llm_answer_gen, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever()
    )

In [ ]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")